# Sujet 2 : GCL RANDU

In [3]:

def normaliser(liste):
    minimum = min(liste)
    maximum = max(liste) 

    newlist = []
    for i in liste :
        newlist.append((i - minimum) / (maximum - minimum))
    return newlist



def GCL_RANDU(nbElement) :  
    Un0 = 12345
    UnPlus = 0
    liste.append(Un0)

    for i in range(nbElement) :
        UnPlus = (65539 * liste[i]) % (2**31)
        liste.append(UnPlus)
    
    return normaliser(liste)


liste = []
liste = GCL_RANDU(10)
print(liste)

[0.0, 0.3861688492024043, 0.26699443596453315, 0.17641849390190023, 0.7055325606669427, 0.5954102596346618, 0.2976375777883839, 0.5270997410478032, 0.4838366770345882, 0.2090939137136397, 1.0]
